# Draw the interaction matrix

The interaction matrix is interactive, requires the ```bokeh``` v3.1.0 python module.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# interaction_matrix = pd.read_csv("D:\\These\\20_data\\212_interaction_matrices\\ADSORPTION_MATRIX_370+host_vs_96_less_stringent.csv", sep=";").set_index("bacteria")
interaction_matrix = pd.read_csv("interaction_matrix.csv", sep=";", na_values="NA").set_index("bacteria")

MODE = "QUANTITATIVE"
print(f"Visualizing... {MODE} interaction matrix!")

phage_features = pd.read_csv("../genomics/phages/guelin_collection.csv", sep=";").set_index("phage").loc[interaction_matrix.columns,]
bact_features = pd.read_csv("../genomics/bacteria/picard_collection.csv", sep=";").set_index("bacteria")

# Impose bacterial phylogenetic order and phage order in heatmap
interaction_matrix.loc["H1-005-0065-L-P"] = np.nan  
interaction_matrix.loc["H27"] = np.nan 
bacteria_order = np.loadtxt("../genomics/bacteria/panacota/tree/370+host_leaf_order_root=albertii.txt", delimiter=",", dtype=str)[::-1]  # load bacteria order

# sort phages
force_phage_order = {"Dhakavirus": 0, "Mosigvirus": 1, "Krischvirus": 2, "Tequatrovirus": 3, "Felixounavirus": 4, "Wifcevirus": 5, "Justusliebigvirus": 6, "Phapecoctavirus": 7, "Gamaleyavirus": 8, "Teseptimavirus": 9, "Przondovirus": 10, "Kayfunavirus": 11, "Vectrevirus": 12, "Veterinaerplatzvirus": 13, "Kagunavirus": 14, "Lambdavirus": 15, "Sashavirus": 16, "Wanchaivirus": 17, "Dhillonvirus": 18}
phage_features["order"] = phage_features["Genus"].apply(lambda x: force_phage_order[x])
phage_features = phage_features.reset_index().rename({"index": "phage"}, axis=1).sort_values(["order", "phage"]).set_index("phage")

interaction_matrix = interaction_matrix.loc[bacteria_order, phage_features.index]  # force phylogenetic ordering in the matrix

print(interaction_matrix.shape, phage_features.shape)

Visualizing... QUANTITATIVE interaction matrix!
(402, 96) (96, 11)


In [9]:
interaction_matrix.replace({5: 4}).astype("uint16").to_csv("interaction_matrix2.csv", sep=";")

In [6]:
from bokeh.models import BasicTicker, PrintfTickFormatter, HoverTool, TextInput, AutocompleteInput, ColumnDataSource, CustomJS
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.io import export_svg, export_png

bacteria = list(interaction_matrix.index)
phages = list(phage_features.index)

# filter phages
phages_of_interest, bact_of_interest = interaction_matrix.columns, interaction_matrix.index
filter_phages, p_condition = False, (phage_features["Phage_host"] == "LF31")#["412_P2", "MT1B1_3A1", "AL505_Sd2"]
filter_bact, b_condition = False, ["H1-001-0053-K-C"]

# reshape to 1D array with one phage-bacteria interaction per line
df = pd.DataFrame(interaction_matrix.loc[bact_of_interest, phages_of_interest].stack(), columns=["score"]).reset_index().rename({"bacteria": "Bacteria", "phage": "Phage"}, axis=1)

df["alpha"] = 1.0
if filter_phages and filter_bact:
    df["alpha"] = 0.1
    p_to_keep = phage_features.loc[p_condition].index
    b_to_keep = bact_features.loc[b_condition].index
    df.loc[df["Phage"].isin(p_to_keep) & df["Bacteria"].isin(b_to_keep), "alpha"] = 1.0
elif not filter_phages and filter_bact:
    df["alpha"] = 0.1
    b_to_keep = bact_features.loc[b_condition].index
    df.loc[df["Bacteria"].isin(b_to_keep), "alpha"] = 1.0
elif filter_phages and not filter_bact:
    df["alpha"] = 0.1
    p_to_keep = phage_features.loc[p_condition].index
    df.loc[df["Phage"].isin(p_to_keep), "alpha"] = 1.0

# bacteria = list(df["Bacteria"].unique())
# phages = ["412_P2", "AL505_Sd2", "MT1B1_3A1"]

# add bacterial and phage genomic features
df = pd.merge(left=df, right=bact_features[["Clermont_Phylo", "ST_Warwick", "LPS_type", "O-type", "H-type", "Klebs_capsule_type", "ABC_serotype"]], left_on="Bacteria", right_index=True)
df = pd.merge(left=df, right=phage_features[["Morphotype", "Genus", "Phage_host"]], left_on="Phage", right_index=True)
df = df.rename({"O-type": "O", "H-type": "H", "Klebs_capsule_type": "Klebs_K", "ABC_serotype": "ABC_K"}, axis=1)

# Choose colormap that suits the interaction matrix type
colors = ["#E1E1EC", "#D4E1EA", "#B1D1E5", "#81B1D2", "#45698A", "#2F3C5A"]  # base blue
colors = ["#ffffff"] + colors

# Display image and genomic features in hover
im_orig_dir = "D:\\These\\20_data\\211_data_ines\\EOP_images_split_by_bact"
source = []
for b in bacteria:
    for p in phages:
        source.append({"Bacteria": b, "Phage": p, "imgs": f"file:///{im_orig_dir}\\{b}\\{b}__{p}.png"})
im_urls = pd.DataFrame(source)
df = pd.merge(df, im_urls, left_on=["Bacteria", "Phage"], right_on=["Bacteria", "Phage"])
hover = """
    <div>
        <div>
            <span style="font-size: 15px; font-weight: bold;">@Bacteria vs. @Phage</span>
        </div>
        <div>
            <span style="font-size: 14px; color: #119; font-weight: bold;">Score: @score</span>
        </div>
        <div>
            <img
                src="@imgs" height="180" alt="@imgs" width="180"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 10px; color: #911;">Phylogroup: @Clermont_Phylo \n</span>
        </div>
        <div>
            <span style="font-size: 10px; color: #911;">ST: @ST_Warwick \n</span>
        </div>
        <div>
           <span style="font-size: 10px; color: #911;">LPS: @LPS_type \n</span>
        </div>
        <div>
           <span style="font-size: 10px; color: #911;">O-type: @O \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #911;">H-type: @H \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #911;">ABC K-type: @ABC_K \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #911;">K-type: @Klebs_K \n</span>
        </div>
        
        <div>
             <span style="font-size: 10px; color: #195;">Morphotype: @Morphotype \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #195;">Genus: @Genus \n</span>
        </div>
        <div>
             <span style="font-size: 10px; color: #195;">Isolation host: @Phage_host \n</span>
        </div>
        
    </div>
"""

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title=f"",
           x_range=bacteria, y_range=phages[::-1],
           x_axis_location="above", width=2500, height=1250, tools=TOOLS,
           tooltips=hover)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi / 3
p.axis.visible = True

r = p.rect(x="Bacteria", y="Phage", width=1, height=1, source=df,
           fill_color=linear_cmap("score", colors, low=df.score.min(), high=df.score.max()),
           line_color="#bbbbbb", line_width=0.10, alpha="alpha")

# show figure or export it in svg format
show(p)